- 导入库

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

- 更换目录

In [2]:
os.chdir('D:\小鸡理财\OneDrive\python')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'D:\\小鸡理财\\OneDrive\\python'

- 导入数据

In [ ]:
with pd.ExcelFile(r'D:\小鸡理财\OneDrive\python\大额用户.xlsx') as xlsx:
    df_hk = pd.read_excel(xlsx,'回款')
    df_cz = pd.read_excel(xlsx,'充值')
    df_tz = pd.read_excel(xlsx,'投资')
    df_md = pd.read_excel(xlsx,'名单')

- 整理数据：保留到日期，筛选出大额用户

- 保留日期的方法

仅保留日期
df['just_date'] = df['dates'].dt.date

保留日期和时间
df['normalised_date'] = df['dates'].dt.normalize()

In [4]:
df_hk['时间'] = pd.to_datetime(df_hk['发放时间'])
df_hk['身份证'] = df_hk['身份证'].str.upper()
df_md['身份证'] = df_md['身份证'].str.upper()

df_hk_ok = df_hk[df_hk['身份证'].isin(df_md['身份证'])]
del df_hk_ok['发放时间']
df_hk_ok.head()

,还款金额,身份证,时间
21,26698.75,330281198711150438,2018-04-04
24,297.50,330281198711150438,2018-04-04
74,50375.00,330227199204162027,2018-04-03
76,201.50,330211197903170069,2018-04-03
85,50375.00,330227199204162027,2018-04-03


In [5]:
df_cz['身份证'] = df_cz['身份证'].str.upper()
df_cz['时间'] = pd.to_datetime(df_cz['成功时间']).dt.date

df_cz_ok = df_cz[df_cz['身份证'].isin(df_md['身份证'])]
del df_cz_ok['成功时间']
df_cz_ok.head()

,充值金额,身份证,时间
3,16000.00,330227196909274922,2018-03-04
4,46.00,330204199105202017,2018-03-04
23,10499.96,330281198711150438,2018-03-04
24,50000.00,330281198711150438,2018-03-04
49,40.00,130406198605270317,2018-03-04


In [6]:
df_tz['身份证'] = df_tz['身份证'].str.upper()
df_tz['时间'] = pd.to_datetime(df_tz['投资时间']).dt.date

df_tz_ok = df_tz[df_tz['身份证'].isin(df_md['身份证'])]
del df_tz_ok['投资时间']
df_tz_ok.head()

,账户资金,身份证,时间
11,26500.00,330281198711150438,2018-03-04
13,34000.00,330281198711150438,2018-03-04
58,50000.00,330227199204162027,2018-03-03
61,200.00,330211197903170069,2018-03-03
67,48245.96,330227199204162027,2018-03-03


- 合并回款、充值、投资

In [7]:
df_hk_gp = df_hk_ok.groupby(['时间','身份证'],as_index=False).sum()
df_cz_gp = df_cz_ok.groupby(['时间','身份证'],as_index=False).sum()
df_tz_gp = df_tz_ok.groupby(['时间','身份证'],as_index=False).sum()

In [8]:
df_hk_gp['时间'] = pd.to_datetime(df_hk_gp['时间'])
df_cz_gp['时间'] = pd.to_datetime(df_cz_gp['时间'])
df_tz_gp['时间'] = pd.to_datetime(df_tz_gp['时间'])

In [9]:
gp_temp = pd.merge(df_hk_gp,df_cz_gp,on = ['时间','身份证'],how = 'outer')
gp = pd.merge(gp_temp,df_tz_gp,on = ['时间','身份证'],how = 'outer')
gp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20153 entries, 0 to 20152
Data columns (total 5 columns):
时间      20153 non-null datetime64[ns]
身份证     20153 non-null object
还款金额    18484 non-null float64
充值金额    2287 non-null float64
账户资金    2614 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 944.7+ KB


- ！扩充每个用户的数据到20180301-20180731的每一天

In [18]:
time = pd.DataFrame({'时间':pd.date_range(start='20180301', end='20180731', freq='D')})
time

#构造一个二级行索引，使得每人对应20180301-20180731的每一天
index = pd.MultiIndex.from_product([time['时间'],df_md['身份证']], names = ['时间','身份证'])
s = pd.Series(1,index=index)

df= s.to_frame().reset_index()
del df[0]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46206 entries, 0 to 46205
Data columns (total 2 columns):
时间     46206 non-null datetime64[ns]
身份证    46206 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 722.0+ KB


,时间,身份证
0,2018-03-01,429001198009252968
1,2018-03-01,330203198106041824
2,2018-03-01,330682199010046315
3,2018-03-01,210302197509053317
4,2018-03-01,62042219840221192X


In [19]:
gp_tm = pd.merge(df,gp,on=['时间','身份证'],how='left')
gp_tm

,时间,身份证,还款金额,充值金额,账户资金
0,2018-03-01,429001198009252968,NaN,NaN,NaN
1,2018-03-01,330203198106041824,NaN,NaN,NaN
2,2018-03-01,330682199010046315,383.16,NaN,NaN
3,2018-03-01,210302197509053317,424.16,5.00,NaN
4,2018-03-01,62042219840221192X,NaN,NaN,NaN
5,2018-03-01,330205196501270946,541.66,NaN,NaN
6,2018-03-01,332621196810203046,49.00,NaN,NaN
7,2018-03-01,610102197802261514,NaN,NaN,NaN
8,2018-03-01,33012319800419001X,2271.25,NaN,NaN
9,2018-03-01,330227196611083023,NaN,NaN,NaN


- 按每个用户计算累加值

In [20]:
gp_tm.fillna(0,inplace=True)

In [21]:
gp_tm['累计还款'] = gp_tm.groupby(['身份证'])['还款金额'].apply(lambda x: x.cumsum())
gp_tm['累计充值'] = gp_tm.groupby(['身份证'])['充值金额'].apply(lambda x: x.cumsum())
gp_tm['累计投资'] = gp_tm.groupby(['身份证'])['账户资金'].apply(lambda x: x.cumsum())

In [22]:
gp_tm.head()

,时间,身份证,还款金额,充值金额,账户资金,累计还款,累计充值,累计投资
0,2018-03-01,429001198009252968,0.00,0.0,0.0,0.00,0.0,0.0
1,2018-03-01,330203198106041824,0.00,0.0,0.0,0.00,0.0,0.0
2,2018-03-01,330682199010046315,383.16,0.0,0.0,383.16,0.0,0.0
3,2018-03-01,210302197509053317,424.16,5.0,0.0,424.16,5.0,0.0
4,2018-03-01,62042219840221192X,0.00,0.0,0.0,0.00,0.0,0.0


In [23]:
gp_tm[gp_tm['身份证']=='130406198605270317']

,时间,身份证,还款金额,充值金额,账户资金,累计还款,累计充值,累计投资
61,2018-03-01,130406198605270317,26.25,0.00,0.00,26.25,0.00,0.00
363,2018-03-02,130406198605270317,16.80,0.00,0.00,43.05,0.00,0.00
665,2018-03-03,130406198605270317,0.00,339.09,339.09,43.05,339.09,339.09
967,2018-03-04,130406198605270317,50395.83,50040.00,50000.00,50438.88,50379.09,50339.09
1269,2018-03-05,130406198605270317,0.00,0.00,0.00,50438.88,50379.09,50339.09
1571,2018-03-06,130406198605270317,103.60,0.00,0.00,50542.48,50379.09,50339.09
1873,2018-03-07,130406198605270317,75656.25,0.00,0.00,126198.73,50379.09,50339.09
2175,2018-03-08,130406198605270317,12912.16,13176.38,13280.92,139110.89,63555.47,63620.01
2477,2018-03-09,130406198605270317,0.00,74913.38,74913.38,139110.89,138468.85,138533.39
2779,2018-03-10,130406198605270317,0.00,0.00,0.00,139110.89,138468.85,138533.39


- 累加值：还款>充值>账户

In [24]:
gp_tm['累计充值_ok'] = np.where(gp_tm['累计还款'] < gp_tm['累计充值'],gp_tm['累计还款'],gp_tm['累计充值'])
gp_tm['累计投资_ok'] = np.where(gp_tm['累计充值_ok'] < gp_tm['累计投资'],gp_tm['累计充值_ok'],gp_tm['累计投资'])

- 按日汇总数据

In [26]:
res = gp_tm.groupby('时间')['累计还款','累计投资_ok'].sum()
res

,累计还款,累计投资_ok
时间,,
2018-03-01,1929871.76,185503.71
2018-03-02,3794937.97,657224.58
2018-03-03,5357580.22,1013945.13
2018-03-04,6052966.69,1216427.47
2018-03-05,7626615.18,1329413.58
2018-03-06,8618113.87,1920812.96
2018-03-07,9885853.73,2210543.30
2018-03-08,10832993.29,2995430.38
2018-03-09,11928018.74,4044344.08


- 导出结果

In [27]:
res.to_excel('大额用户经典版回款新充投资比.xlsx')